# 1. Techniques for the generation of Cosmic Microwave Background maps

### Introduction from Jeff McMahon and Renée Hložek for the TMCL - CMB school 2019

<i><font color='red'>NOTE</font>: I've changed some redundant information in all texts throughout the notebooks, but left relevant informations unchanged.</i>

The Cosmic Microwave Background (CMB) is the oldest observable light in the universe. As such it carries a wealth of cosmological information including: (1) signals from the early universe (primary anisotropy), and (2) distortions imprinted as this light propagates through the universe and encounters collapsed structures (secondary anisotropy). Measurements of these signals give us important measurements and constraints on inflationary parameters, dark energy, dark matter, the sum of the neutrino masses, and many astrophysical processes. The development of CMB instruments and analysis techniques is rapidly evolving.

This notebook provides an introduction to analysis techniques for CMB maps as they pertain to high resolution CMB instruments such as the Atacama Cosmology Telescope (ACT) and the South Pole Telescope (SPT). These state of the art instruments have thousands of detectors (pixels) deployed on 6 and 10 meter telescopes which produce arc minute resolution beams at 150 GHz. These telescopes observe by scanning across the sky. The data from each detector are read out at high speed (typically > 50 Hz) to produce 'detector time streams' which are recored along with pointing information and other ancillary data. Sophisticated codes are used to convert these time streams into maps of the CMB sky. These maps can have correlated noise that must be accounted for in data analysis. The mapmaking algorithms are not described here but represent a very interesting big data challenge as they require cleaning time streams by filtering, identifying transient events, and ultimately compressing ~Tb of data down to maps that are typically 100 Mb or less.  

Here are example maps from SPT (top, taken from a phys.org article from April 2, 2012 titled "South Pole Telescope hones in on dark energy, neutrinos") and ACTPol (bottom, taken from the Naess et al. 2014 CMB polarization paper). Many features are obvious in these maps including: (1) the primary CMB visible as waves in the intensity, (2) active galactic nuclei and other bright astrophysical point sources which manifest as bright dots, (3) clusters of galaxies which show up as darkened point sources, and (4) horizontal stripes (in the SPT map) that are the result of the noise and map filtering. The ACTPol figure shows multiple maps; the T is temperature, Q and U are polarization, and E and B are also polarization but decomposed into a basis such that the E is the amplitude of the curl-free component, and B is the amplitude of the divergence free component of the polarization vector field.

![cosmo_img](./img/newcosmology.jpg "Cosmology")

![cmb_maps](./img/maps_b.png "CMB")

While the current instruments (ACTPol and SPTPol) have multiple frequencies and polarization sensitivity, for simplicity we consider only a single frequency (150 GHz) and only temperature. Adding extra frequency channels or polarization add the cost of slightly more work and would make this presentation more confusing. Therefore we leave these complications for a future treatment and focus on temperature alone. *[This allows us to show the basics of Monte-Carlo analysis of both the angular power spectrum and matched filter techniques for studying Sunyaev-Zeldovich (SZ) effect (<font color='red'>NOTE</font>: This latter part isn't included in my project.)]*

The short course is organized as follows. We beginning by building simulation of the CMB sky including the primary CMB, point sources (AGN and dusty galaxies), the SZ effect, and (if time) CMB lensing. To these maps, we fold in instrumental effects including the beam, instrument and atmospheric noise. *[We then present several analysis techniques including Monte-Carlo estimation of power spectra and matched filter techniques for extraction of sources. An example of a stacking analysis is presented as an example of a cross-correlation with external data sets. Cross-correlation is a very active field of research (<font color='red'>NOTE</font>: This latter part isn't included in my project.)]*

In the interest of simplicity we use approximate distributions for source and SZ counts with combinations of exponential and poisson distributions.   We note explicitly where we make these approximations.

## Code preliminaries

In [ ]:
import os
import sys
import numpy as np

import seaborn as sns
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.colors import ListedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
from constants import *
from cmb_modules import *

In [ ]:
data = '../data/'
out = '../output/'

### Just some matplotlib and seaborn parameter tuning

In [ ]:
axistitlesize = 20
axisticksize = 17
axislabelsize = 26
axislegendsize = 23
axistextsize = 20
axiscbarfontsize = 15

# Set axtick dimensions
major_size = 6
major_width = 1.2
minor_size = 3
minor_width = 1
mpl.rcParams['xtick.major.size'] = major_size
mpl.rcParams['xtick.major.width'] = major_width
mpl.rcParams['xtick.minor.size'] = minor_size
mpl.rcParams['xtick.minor.width'] = minor_width
mpl.rcParams['ytick.major.size'] = major_size
mpl.rcParams['ytick.major.width'] = major_width
mpl.rcParams['ytick.minor.size'] = minor_size
mpl.rcParams['ytick.minor.width'] = minor_width

mpl.rcParams.update({'figure.autolayout': False})

# Seaborn style settings
sns.set_style({'axes.axisbelow': True,
               'axes.edgecolor': '.8',
               'axes.facecolor': 'white',
               'axes.grid': True,
               'axes.labelcolor': '.15',
               'axes.spines.bottom': True,
               'axes.spines.left': True,
               'axes.spines.right': True,
               'axes.spines.top': True,
               'figure.facecolor': 'white',
               'font.family': ['sans-serif'],
               'font.sans-serif': ['Arial',
                'DejaVu Sans',
                'Liberation Sans',
                'Bitstream Vera Sans',
                'sans-serif'],
               'grid.color': '.8',
               'grid.linestyle': '--',
               'image.cmap': 'rocket',
               'lines.solid_capstyle': 'round',
               'patch.edgecolor': 'w',
               'patch.force_edgecolor': True,
               'text.color': '.15',
               'xtick.bottom': True,
               'xtick.color': '.15',
               'xtick.direction': 'in',
               'xtick.top': True,
               'ytick.color': '.15',
               'ytick.direction': 'in',
               'ytick.left': True,
               'ytick.right': True})

# Colorpalettes, colormaps, etc.
sns.set_palette(palette='rocket')

## Section 1.1: Angular Power Spectrum
The majority of the information content of the CMB is contained in its angular power spectrum.   This spectrum is the amplitude squared of the magnitude of the temperature fluctuations as a function of $\ell$.  Here $\ell$ is the variable you have seen before with the spherical harmonics (e.g., $Y_{\ell m}$).  $\ell = 0$ corresponds to a constant temperature across the sky, $\ell = 200$ corresponds approximately to scales of $1^\circ$.  For a given set of input cosmological parameters these spectra can be computed with codes including CMBFAST or the more modern equivalent CAMB.  The spectrum used below was computed with CAMB web interface on NASA's LAMBDA interface.

http://lambda.gsfc.nasa.gov/toolbox/tb_camb_form.cfm

It is fun to play with parameters and see how the spectrum changes. 

The relative locations and amplitudes of the peaks carry a great deal of information.  The location of the first peak at $\ell \sim 200$ provided the first definitive measurement that our universe is flat.  The shape of the spectrum constraints a wide variety of cosmological parameters including dark energy and dark matter.  The fall off at high ell (e.g. $\ell \gtrsim 1000$ is called the damping tail and it provides constrains on the spectral index from inflation.  A wealth of cosmological parameters are constrained by measurements of this spectrum.  

At this point the temperature spectrum is well measured and the open frontiers are polarization and secondary anisotropies that are not included in this spectrum.   We will now build simulated skies including the CMB anisotropy and secondary anisotropies.  After that we will show how to estimate the power spectrum from map data using monte carlo techniques.   This monty carlo approach can be used for interpretation of polarization data and for the interpretation of cross-correlations between CMB and other survey data.

Here is how to read in an plot the CMB temperature spectrum from a CAMB simulation.

In [ ]:
# Read in the input CMB spectra
d = np.genfromtxt(data + 'CAMB_real_scalCls.dat')
ell = d[:,0]
DlTT = d[:,1]
ell_err = d[:,2]
DlTT_err = d[:,3]

In [ ]:
fig, axes = plt.subplots(figsize=(15, 9),
                         facecolor='black', subplot_kw={'facecolor' : 'black'})
cut = 2500
axes.plot(ell[:cut], DlTT[:cut],
          color=cm.magma(0.93), lw=3)

axes.set_xlabel('$\ell$', fontsize=axislabelsize, fontweight='bold', color='white')
axes.set_ylabel('$D_{\ell}$ [$\,\mu$K$^2\,$]', fontsize=axislabelsize, fontweight='bold', color='white')
axes.tick_params(axis='both', which='major', labelsize=axisticksize, colors='white')

fig.suptitle('Fig. 1. $D_{\ell}$ spectra of CMB',
             color='white', fontsize=axistitlesize, y=0.04)

plt.savefig(out + 'cmb_angular_spectrum.png',
            format='png', dpi=200,
            facecolor='black', edgecolor='black',
            bbox_inches='tight')

plt.show()

This plot is the angular power spectrum of the CMB temperature anisotropies. It is traditional to plot $D_\ell$ which give the variance on varius angular scales. This is related to the angular power spectrum $C_\ell$ by

$$
D_\ell = \frac {\ell (\ell+1)} {2 \pi} C_\ell.
$$

The correct thing to use for creating CMB maps is $C_\ell$, but $D_\ell$ is what you are used to seeing plotted.

Interesting <font color='red'>EXCERCISE</font> from the original CMB school notebooks: Go to the NASA LAMBDA CAMB web interface and generate a power spectrum for a for a model universe with a strange component of your choosing (e.g., with spatial curvature, no dark energy, some other bizare component). Make a plot comparing this power spectrum to the vanilla spectrum above.

In [ ]:
# Read in the input CMB spectra
d = np.genfromtxt(data + 'CAMB_strange_scalCls.dat')
ell_strange = d[:,0]
DlTT_strange = d[:,1]

In [ ]:
fig, axes = plt.subplots(figsize=(15, 9),
                         facecolor='black', subplot_kw={'facecolor' : 'black'})
cut = 2500
axes.plot(ell[:cut], DlTT[:cut], label='True cosmology',
          color=cm.magma(0.93), lw=3, ls='--')
axes.plot(ell_strange[:cut], DlTT_strange[:cut], label='Artificial cosmology',
          color=cm.magma(0.75), lw=3)

axes.set_xlabel('$\ell$', fontsize=axislabelsize, fontweight='bold', color='white')
axes.set_ylabel('$D_{\ell}$ [$\,\mu$K$^2\,$]', fontsize=axislabelsize, fontweight='bold', color='white')
axes.tick_params(axis='both', which='major', labelsize=axisticksize, colors='white')

axes.legend(loc='upper right', fontsize=axislegendsize)

fig.suptitle('Fig. 2. $D_{\ell}$ spectra of CMB with $\\Omega_{\\nu} = 0.1$ compared to the CMB\n' +
             'spectrum generated using true cosmological parameters.',
             color='white', fontsize=axistitlesize, y=0.04)

plt.savefig(out + 'cmb_angular_spectrum.png',
            format='png', dpi=200,
            facecolor='black', edgecolor='black',
            bbox_inches='tight')

plt.show()

## Section 1.2: Temperature Anisotropy Map
In this step we generate a simulated map of the CMB sky with the spectrum we read in above.  Since the power spectrum is a function of $\ell$ we need to do much of the work in harmonic space.  If we were generating a map on the full sky we would need to work with spherical harmonics.  Here we consider a small patch of sky ($\sim 10^\circ \times 10^\circ$) were we cam is the 'flat-sky' approximation and replace $\ell$ with $k = \sqrt{k_{x}^{2} + k_{y}^{2}}$.  There is a linear dependance between these variables defined by $\ell = k \cdot 2 \pi$.

In the flat sky approximation we generate a CMB map by:
    
    (1) generating a 2D power spectrum by revolving the above spectrum (properly normalized) about the axis in polar coordinates,
    (2) generating a Gaussian random map with unit variance as our 'random realisation',
    (3) multiplying that maps from 1 and 2, and
    (4) Fourier transforming this to get a real space map.

We provide a function to do this and a function to plot this (and other maps) with a uniform color scale.

In [ ]:
ell2d, ClTT2d, FT_2d, CMB_I = make_CMB_I_map(ell, DlTT, N_x=N_x, N_y=N_y,
                                             X_width=X_width, Y_width=Y_width, pix_size=pix_size,
                                             random_seed=69)

In [ ]:
plot_CMB_map(CMB_I, X_width, Y_width, c_min=c_min, c_max=c_max,
             save=True, save_filename='CMB_sim_pure_true.png',
             no_axis=False, no_grid=True, no_title=False,
             no_cbar=False)

This contour plot shows simulated CMB map we just generated.  If you generate it multiple times you will find that you get different patterns, however you will see that the typical size of the brightest and darkest spots will stay around $1^\circ$, corresponding to the peak of the angular power spectrum.  All simulated sky maps are displayed with the same color scale to permit visual comparison.

In [ ]:
# Create the the 2D CMB simulated map in Fourier space, note the x and y axis labels need to be fixed
CMB_2D = np.real(np.conj(FT_2d) * FT_2d * ell2d * (ell2d + 1) / (2 * np.pi))

In [ ]:
plot_steps_2D_ps(ClTT2d,
                 X_width, Y_width,
                 no_axis=False, no_grid=True)

In [ ]:
plot_steps_2D_gauss(ell2d, FT_2d, CMB_2D,
                    X_width, Y_width,
                    no_axis=False, no_grid=True)

## Section 1.3: Strange and interesting CMB maps

Visualizing the generated strange cosmology or cutting the power spectrum at a given multipole results interesting CMB maps. Explore some of them here.

### Cut spectrum below $\ell < 1000$

In [ ]:
DlTT_cb = DlTT.copy()
DlTT_cb[ell < 1000] = 0
ell2d, ClTT2d, FT_2d, CMB_I = make_CMB_I_map(ell, DlTT_cb, N_x=N_x, N_y=N_y,
                                             X_width=X_width, Y_width=Y_width, pix_size=pix_size)

In [ ]:
plot_CMB_map(CMB_I, X_width, Y_width, c_min=-60, c_max=60,
             save=False,
             no_axis=False, no_grid=True, no_title=False,
             no_cbar=False)

In [ ]:
# Create the the 2D CMB simulated map in Fourier space, note the x and y axis labels need to be fixed
CMB_2D = np.real(np.conj(FT_2d) * FT_2d*ell2d * (ell2d+1) / (2*np.pi))

In [ ]:
plot_steps_2D_ps(ClTT2d,
                 X_width, Y_width,
                 no_axis=False, no_grid=True)

In [ ]:
plot_steps_2D_gauss(ell2d, FT_2d, CMB_2D,
                    X_width, Y_width,
                    no_axis=False, no_grid=True)

### Cut spectrum above $\ell > 1000$

In [ ]:
DlTT_ca = DlTT.copy()
DlTT_ca[ell > 1000] = 0
ell2d, ClTT2d, FT_2d, CMB_I = make_CMB_I_map(ell, DlTT_ca, N_x=N_x, N_y=N_y,
                                             X_width=X_width, Y_width=Y_width, pix_size=pix_size)

In [ ]:
plot_CMB_map(CMB_I, X_width, Y_width, c_min=c_min, c_max=c_max,
             save=False,
             no_axis=False, no_grid=True, no_title=False,
             no_cbar=False)

In [ ]:
# Create the the 2D CMB simulated map in Fourier space, note the x and y axis labels need to be fixed
CMB_2D = np.real(np.conj(FT_2d) * FT_2d*ell2d * (ell2d+1) / (2*np.pi))

In [ ]:
plot_steps_2D_ps(ClTT2d,
                 X_width, Y_width,
                 no_axis=False, no_grid=True)

In [ ]:
plot_steps_2D_gauss(ell2d, FT_2d, CMB_2D,
                    X_width, Y_width,
                    no_axis=False, no_grid=True)

### Strange cosmology

Saving both map generated using the true and this strange cosmology and comparing them is interesting I think.

In [ ]:
ell2d, ClTT2d, FT_2d, CMB_I = make_CMB_I_map(ell_strange, DlTT_strange, N_x=N_x, N_y=N_y,
                                             X_width=X_width, Y_width=Y_width, pix_size=pix_size,
                                             random_seed=42)

In [ ]:
plot_CMB_map(CMB_I, X_width, Y_width, c_min=c_min, c_max=c_max,
             save=False, save_filename='CMB_sim_pure_strange.png',
             no_axis=False, no_grid=True, no_title=False,
             no_cbar=False)

In [ ]:
# Create the the 2D CMB simulated map in Fourier space, note the x and y axis labels need to be fixed
CMB_2D = np.real(np.conj(FT_2d) * FT_2d*ell2d * (ell2d+1) / (2*np.pi))

In [ ]:
plot_steps_2D_ps(ClTT2d,
                 X_width, Y_width,
                 no_axis=False, no_grid=True)

In [ ]:
plot_steps_2D_gauss(ell2d, FT_2d, CMB_2D,
                    X_width, Y_width,
                    no_axis=False, no_grid=True)